# Import libraries

In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

In [2]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Basic libraries
#
import json
import pandas   as pd
import requests


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# User libraries
#
from utils.Logger   import *

# Parameters

In [3]:
# Other parameters
#
VERBOSE = True

In [4]:
# Initiate logger
#
if VERBOSE:
    logger = init_logger( log_file = 'logs.log' ) 

# Import data

**Dataset**

- Irrigation 


**Context**

The scope is to predict if a region is 'irrigated' or 'drainaged' based on satellite multi-temporal data (indices)

# Loading data

In [5]:
df = pd.read_csv('Data/Irrigation_train.csv')

if VERBOSE:
    logger.info(f'Data were loaded')
    logger.info(f'Number of instances:  {df.shape[0]}')
    logger.info(f'Number of features:   {df.shape[1]}')

df.head( 3 )

[INFO] Data were loaded
[INFO] Number of instances:  82503
[INFO] Number of features:   29


,B11_5,B12_5,B05_5,B06_5,NDVI_5,NDBI_5,LSWI2_5,B11_7,B12_7,B05_7,...,NDBI_9,LSWI2_9,B11_11,B12_11,B05_11,B06_11,NDVI_11,NDBI_11,LSWI2_11,Irrigation
0,0.325158,0.211388,0.162878,0.199771,0.163840,-0.267323,0.014312,0.324716,0.204250,0.174650,...,-0.264708,0.041466,0.315360,0.201311,0.161962,0.191630,0.159782,-0.262815,0.039018,0
1,0.283810,0.185874,0.204715,0.292734,0.321123,-0.359163,0.263899,0.283403,0.185829,0.195995,...,-0.353670,0.287560,0.262824,0.169689,0.170315,0.270060,0.288574,-0.334296,0.248330,1
2,0.323059,0.200238,0.199566,0.288144,0.347120,-0.392760,0.259642,0.350821,0.223944,0.210791,...,-0.439905,0.268981,0.323004,0.195420,0.183496,0.256162,0.310311,-0.361256,0.211741,1


## Initiate MLFlow server

In [6]:
# Initiate mlflow server
# Command: mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts --host 0.0.0.0 --port 5000
# 
import mlflow

mlflow.set_tracking_uri("http://0.0.0.0:5000/")
mlflow.set_experiment("Irrigation-Experiment")


if VERBOSE:
    logger.info('MLFlow server is connected')

[INFO] MLFlow server is connected


# Serving a model with MLflow 

## Serve the production model from the model registry (Instructions)

In [7]:
# # > Serving an MLflow Model from Model Registry
# #
# #
# # Set environment variable for the tracking URL where the Model Registry resides
# export MLFLOW_TRACKING_URI=http://0.0.0.0:5000
# #
# # Serve the production model from the model registry
# mlflow models serve -m "models:/Irrigation_model/Production" --no-conda --port 1983

## Create an instance

In [8]:
d = {
  "B11_5": 0.21,
  "B12_5": 0.32,
  "B05_5": 0.123,
  "B06_5": 0.141,
  "NDVI_5": 0.131,
  "NDBI_5": 0.123,
  "LSWI2_5": 0.456,
  "B11_7": 0.456,
  "B12_7": 0.345,
  "B05_7": 0.687,
  "B06_7": 0.234,
  "NDVI_7": 0.012,
  "NDBI_7": 0.123,
  "LSWI2_7": 0.415,
  "B11_9": 0.513,
  "B12_9": 0.561,
  "B05_9": 0.941,
  "B06_9": 0.151,
  "NDVI_9": 0.614,
  "NDBI_9": 0.726,
  "LSWI2_9": 0.724,
  "B11_11": 0.161,
  "B12_11": 0.614,
  "B05_11": 0.613,
  "B06_11": 0.417,
  "NDVI_11": 0.745,
  "NDBI_11": 0.978,
  "LSWI2_11": 0.666
   }

# Convert to suitable form
for x in d.keys():
  d[x] = [ d[x] ]

In [9]:
url     = 'http://127.0.0.1:1983/invocations'
headers = {'Content-Type': 'application/json; format=pandas-records'}

try:
    # Provide inference for custom instance
    #
    response = requests.post(url    = url, 
                            data    = json.dumps(d), 
                            headers = headers)

    # # Provide inference for DataFrame
    # #
    # response = requests.post(url    = url, 
    #                         data    = pd.DataFrame.to_json( df[:5] ), 
    #                         headers = headers)    

    print('Status: ', response.status_code)
    print(response.content)
except Exception as ex:
    raise (ex)

Status:  200
b'[1]'
